In [47]:
import time
import numpy as np
from copy import deepcopy

In [48]:
class State:

    def __init__(self, state, pre_move = 'root'):
        self.state = np.array([state[:3], state[3:6], state[6:]])
        self.pre_move = pre_move
        self.FindZero()

    def __eq__(self, other):
        if isinstance(other, State):
            return np.array_equal(self.state, other.state)
        
    def __hash__(self):
        return hash(tuple(map(tuple, self.state)))

    def FindZero(self):
        index = np.where(self.state == 0)
        coor = list(zip(index[0], index[1]))
        return list(coor[0])
    
    def ValidMove(self, move):
        if move == 'up':
            if self.FindZero()[0] != 0:
                return True
        elif move == 'down':
            if self.FindZero()[0] != 2:
                return True
        elif move == 'left':
            if self.FindZero()[1] != 0:
                return True
        elif move == 'right':
            if self.FindZero()[1] != 2:
                return True
        else:
            print('invalid move')
            return False
    
    def SavedMove(self, move):
        if self.ValidMove(move):
            saved = deepcopy(self)
            row = self.FindZero()[0]
            col = self.FindZero()[1]
            if move == 'up':
                self.state[row][col] = self.state[row-1][col]
                self.state[row-1][col] = 0
                self.pre_move = 'up'
            elif move == 'down':
                self.state[row][col] = self.state[row+1][col]
                self.state[row+1][col] = 0
                self.pre_move = 'down'
            elif move == 'left':
                self.state[row][col] = self.state[row][col-1]
                self.state[row][col-1] = 0
                self.pre_move = 'left'
            elif move == 'right':
                self.state[row][col] = self.state[row][col+1]
                self.state[row][col+1] = 0
                self.pre_move = 'right'
            return saved
        else:
            return False
        
    def Expand(self):
        nodes = []
        action_list = ['up','down','left','right']
        loop_move = ['down','up','right','left']
        for action in range(0,4):
            if(self.ValidMove(action_list[action]) and self.pre_move != loop_move[action]):
                temp = deepcopy(self)
                temp.SavedMove(action_list[action])
                nodes.append(temp)
        return nodes

In [49]:
def DFS(state, goal, depth, path):
    if state == goal:
        path.append(state.pre_move)
        explored[state] = True
        return True
    else:
        if depth == 0:
            explored[state] = True
            return False
        else:
            explored[state] = True
            for node in state.Expand():
                if node in explored:
                    continue
                else:
                    found = DFS(node, goal, depth-1, path)
                    if found:
                        path.append(state.pre_move)
                        return True
            return False
        
def IterativeDeepening(start, goal, path):
    global explored
    i = 0
    found = False
    while len(explored) <= 1000000:
        explored = {}
        path = []
        DFS(start, goal, i, path)
        i += 1
        if (len(path) != 0):
            print('Depth: ',i)
            path.reverse()
            print('Path: ', path)
            print('Number of moves used: ', len(path)-1)
            print('Number of nodes expanded: ', len(explored))
            found = True
            break
    if not found:
        print('Limit reached, solution not found')

In [50]:
def InputToArray(s):
    a = []
    for c in s:
        a.append(int(c))
    return a

def ValidInput(a):
    checker = [0,1,2,3,4,5,6,7,8]
    for i in a:
        if i in checker:
            checker.remove(i)
        else:
            return False
    if checker:
        return False
    return True

In [51]:
if __name__ == '__main__':
    
    start_input = input('Please Enter the Start state of the puzzle (e.g. 724506831 for the start state): ')
    goal_input = input('Please Enter the goal state of the puzzle (e.g. 012345678 for the goal state): ')

    if start_input.isnumeric() and goal_input.isnumeric():
        start = InputToArray(start_input)
        goal = InputToArray(goal_input)
    else:
        print('Invalid Input for start state or goal state')
        print('Start state: ', start_input)
        print('Goal state: ', goal_input)
        raise

    explored = {}
    path = []
    
    if ValidInput(start) and ValidInput(goal):
        start_state = State(start)
        goal_state = State(goal)
        start_time = time.perf_counter()
        #Run time is about 3 mintues
        IterativeDeepening(start_state, goal_state, path)
        end_time = time.perf_counter()
        print(f'Algorithm Run Time: {end_time - start_time} seconds')
    else:
        print('Invalid Input for start state or goal state')
        print('Start state: ', start_input)
        print('Goal state: ', goal_input)
        raise

Depth:  35
Path:  ['root', 'right', 'up', 'left', 'left', 'down', 'right', 'down', 'left', 'up', 'right', 'up', 'right', 'down', 'down', 'left', 'up', 'up', 'right', 'down', 'left', 'up', 'left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up']
Number of moves used:  30
Number of nodes expanded:  68907
Algorithm Run Time: 37.97188130003633 seconds
